In [ ]:
from langchain_lama import OllamaLLM # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore

template = """

Answer the question below.

Here is the conversation history: {context}

Question: {question}
he
Answer:

"""

model = OllamaLLM(model="llama3.2")
promt = ChatPromptTemplate.from_template(template)
chain = promt | model

# def handle_convo():
#     context = ""
#     print("Welcome to the AI Chatbot! Type 'exit' to quit.")
#     while True:
#         user_input = input("You: ")
#         if user_input.lower() == "exit":  
#             break
#         result = chain.invoke({"context": context, "question": user_input})
#         print("AI: ", result)
#         context += f"\nUser: {user_input}\nAI: {result}"

# if __name__ == "__main__":
#     handle_convo()


import streamlit as st

st.title('Llama 3.2')

if 'message' not in st.session_state:
    st.session_state.message = []
    st.session_state.context = ""
    
for message in st.session_state.message:
    st.chat_message(message['role']).markdown(message['content'])

prompt = st.chat_input('Pass your prompt here')

if prompt:
    st.chat_message('user').markdown(prompt)
    st.session_state.message.append({'role':'user', 'content':prompt})
    st.session_state.context += f"\nUser: {prompt}\n"
    result = chain.invoke({"context": st.session_state.context, "question": prompt})
    st.chat_message('assistant').markdown(result)
    st.session_state.message.append({'role':'assistant', 'content':result})
    st.session_state.context += f"AI: {result}\n"

In [ ]:
import aiohttp
import asyncio
import json
import os
from dotenv import load_dotenv

load_dotenv()

async def invoke_chute():
    api_token = os.getenv("chutes_api")  

    headers = {
        "Authorization": "Bearer " + api_token,
        "Content-Type": "application/json"
    }
    
    body = {
        "model": "deepseek-ai/DeepSeek-V3-0324",
        "messages": [
            {
                "role": "user",
                "content": "Tell me a 100 word story."
            }
        ],
        "stream": True,
        "max_tokens": 1024,
        "temperature": 0.7
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(
            "https://llm.chutes.ai/v1/chat/completions", 
            headers=headers,
            json=body
        ) as response:
            async for line in response.content:
                line = line.decode("utf-8").strip()
                if line.startswith("data: "):
                    data = line[6:]
                    if data == "[DONE]":
                        break
                    try:
                        chunk = json.loads(data.strip())
                        if "choices" in chunk and chunk["choices"]:
                            delta = chunk["choices"][0].get("delta", {})
                            if "content" in delta:
                                print(delta["content"], end="", flush=True)
                    except Exception as e:
                        print(f"Error parsing chunk: {e}")

await invoke_chute()